# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.118047e+02     1.678916e+00
 * time: 0.1271660327911377
     1     1.098278e+01     1.036426e+00
 * time: 0.9389228820800781
     2    -1.177605e+01     1.247153e+00
 * time: 1.0701611042022705
     3    -3.414738e+01     8.672077e-01
 * time: 1.2296490669250488
     4    -4.771223e+01     6.306280e-01
 * time: 1.3788509368896484
     5    -5.705415e+01     1.966517e-01
 * time: 1.521320104598999
     6    -5.998735e+01     8.855971e-02
 * time: 1.6260221004486084
     7    -6.096803e+01     7.395612e-02
 * time: 1.737795114517212
     8    -6.137245e+01     6.958022e-02
 * time: 1.8436920642852783
     9    -6.169933e+01     4.061100e-02
 * time: 1.9583220481872559
    10    -6.190526e+01     3.095862e-02
 * time: 2.065433979034424
    11    -6.203424e+01     2.133797e-02
 * time: 2.177988052368164
    12    -6.209715e+01     1.772219e-02
 * time: 2.2829771041870117
    13    -6.214429e+01     1.323183e-02
 * time: 2.39684104919433

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671068
    AtomicLocal         -18.8557671
    AtomicNonlocal      14.8522636
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485375 
    Xc                  -19.3336822

    total               -62.261666458759
